<a href="https://colab.research.google.com/github/BIRAN/SAS_Visual_Forecasting/blob/master/SARIMA_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pandas import read_csv
import pandas as pd
import numpy as np
import time
from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
import requests
from io import BytesIO

%matplotlib inline

In [0]:
def parser(x):
    return datetime.strptime(x, '%Y-%m-%d')

url = 'https://raw.githubusercontent.com/BIRAN/SAS_Visual_Forecasting/master/statistical_method/air_date.csv'
series = pd.read_csv(url, header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

In [3]:
series

DATE
1949-01-01    112
1949-02-01    118
1949-03-01    132
1949-04-01    129
1949-05-01    121
1949-06-01    135
1949-07-01    148
1949-08-01    148
1949-09-01    136
1949-10-01    119
1949-11-01    104
1949-12-01    118
1950-01-01    115
1950-02-01    126
1950-03-01    141
1950-04-01    135
1950-05-01    125
1950-06-01    149
1950-07-01    170
1950-08-01    170
1950-09-01    158
1950-10-01    133
1950-11-01    114
1950-12-01    140
1951-01-01    145
1951-02-01    150
1951-03-01    178
1951-04-01    163
1951-05-01    172
1951-06-01    178
             ... 
1958-07-01    491
1958-08-01    505
1958-09-01    404
1958-10-01    359
1958-11-01    310
1958-12-01    337
1959-01-01    360
1959-02-01    342
1959-03-01    406
1959-04-01    396
1959-05-01    420
1959-06-01    472
1959-07-01    548
1959-08-01    559
1959-09-01    463
1959-10-01    407
1959-11-01    362
1959-12-01    405
1960-01-01    417
1960-02-01    391
1960-03-01    419
1960-04-01    461
1960-05-01    472
1960-06-01    535
1960-

In [4]:
# SARIMA model test
# 12/03/2018

# International Airline Passengers
# dataset is the Box-Jenkins Series-G
# we talk the log, b/c it is multiplicative seasonality
data= series
df = np.log(data)

df1=df.diff(periods=1)
df1_12=df1.diff(periods=12)
df1_12.mean()     # same as SAS proc arima output
df1_12.std()      # same as SAS proc arima output
data= df1_12


my_order = (0, 1, 1)
my_seasonal_order = (0, 1, 1, 12)
number_iteration = 10


# Aim: calculate the running time
start_time = time.time()
for i in range(0,number_iteration):
    sm.tsa.statespace.SARIMAX(df, order=my_order, seasonal_order=my_seasonal_order).fit().aic
    i=i+1


time.time() - start_time

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.6/dist-

7.586121082305908

In [5]:
model = sm.tsa.statespace.SARIMAX(df, order=my_order, seasonal_order=my_seasonal_order)
model_fit = model.fit()
print(model_fit.summary())

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)


                                 Statespace Model Results                                 
Dep. Variable:                                AIR   No. Observations:                  144
Model:             SARIMAX(0, 1, 1)x(0, 1, 1, 12)   Log Likelihood                 244.686
Date:                            Mon, 12 Aug 2019   AIC                           -483.371
Time:                                    18:42:19   BIC                           -474.746
Sample:                                01-01-1949   HQIC                          -479.866
                                     - 12-01-1960                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.4140      0.073     -5.666      0.000      -0.557      -0.271
ma.S.L12      -0.5518      0.097   

In [0]:
model?

In [0]:
# Two ways to make predictions

forecast_horizon1 = model_fit.forecast(12)

forecast_horizon2 = model_fit.predict(start=144, end=156)

In [0]:
# b/c we took log at the beginning
# we need to transform it back
predictions = np.exp(forecast_horizon)